**Sentiment Analysis of IMDB Movie Reviews**

**Problem Statement:**

We have to predict the number of positive and negative reviews(Sentiment Analysis) based on Review text by using NLP techniques and different classification models.

**Import necessary libraries**

In [1]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\611242028\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import os 
path = os.getcwd()
print(path)

C:\Users\611242028\Data Science\Projects


**Import the training dataset**

In [3]:
#importing the training data
imdb_data=pd.read_csv('IMDB_Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


**Exploratery data analysis**

In [4]:
#Summary of the dataset
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


**Sentiment count**

In [5]:
#sentiment count
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

We can see that the dataset is balanced.

**Text normalization**

In [6]:
#Tokenization of text object
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

**Removing html strips and noise text**

In [7]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

**Removing special characters**

In [8]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_special_characters)

**Text stemming**

In [9]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(simple_stemmer)

**Removing stopwords**

In [10]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(remove_stopwords)

{"mustn't", 'were', "couldn't", 'does', 'or', "wasn't", 'not', 'all', 'into', 'those', 'having', 'did', 'it', 'ain', "you'll", 'some', 'should', 'against', 'we', 'weren', "don't", 'after', 'because', 'about', 'o', 'each', 'me', 'him', 'himself', 'she', 'shouldn', 'wasn', "it's", "doesn't", 'if', 'again', 'themselves', 'couldn', "shan't", 'over', 'are', 'down', 'too', 'd', 'shan', 'her', 'while', 'hasn', 'which', 'at', 'as', 'where', 'further', 'can', 'yours', 'will', 'he', 'this', 'do', 'with', 'our', 'before', 'own', 'when', 'why', "isn't", "needn't", "you've", 'm', 'yourselves', 'them', 'above', 'how', 'just', 'other', 'only', 'until', 'ourselves', 'be', 'under', "didn't", 'than', "hadn't", "should've", 'between', 'very', 'hadn', "mightn't", 'once', 'aren', 'for', 'any', 'was', 'have', 'his', 'won', 'to', 'didn', "haven't", 'wouldn', 't', 'doing', 'being', 'll', "she's", 'their', 'below', 'both', 'through', 'needn', "that'll", "you're", "shouldn't", "hasn't", "weren't", 'so', 'haven'

**Labelling the target into Binary Numeric Format**

In [11]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(imdb_data['sentiment'])
print(sentiment_data.shape)

(50000, 1)


**Splitting train and test data**

In [12]:
#split the dataset  using the train_test split library
X= imdb_data.review
y= sentiment_data
train_reviews, test_reviews, train_sentiments, test_sentiments =train_test_split(X,y,test_size=0.25,random_state=42)
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(37500,) (37500, 1)
(12500,) (12500, 1)


In [13]:
train_reviews[0]

'one review ha mention watch 1 Oz episod youll hook right thi exactli happen meth first thing struck Oz wa brutal unflinch scene violenc set right word GO trust thi show faint heart timid thi show pull punch regard drug sex violenc hardcor classic use wordit call OZ nicknam given oswald maximum secur state penitentari focus mainli emerald citi experiment section prison cell glass front face inward privaci high agenda Em citi home manyaryan muslim gangsta latino christian italian irish moreso scuffl death stare dodgi deal shadi agreement never far awayi would say main appeal show due fact goe show wouldnt dare forget pretti pictur paint mainstream audienc forget charm forget romanceoz doesnt mess around first episod ever saw struck nasti wa surreal couldnt say wa readi watch develop tast Oz got accustom high level graphic violenc violenc injustic crook guard wholl sold nickel inmat wholl kill order get away well manner middl class inmat turn prison bitch due lack street skill prison exp

**Bags of words model **

It is used to convert text documents to numerical vectors or bag of words.

In [14]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (37500, 5866919)
BOW_cv_test: (12500, 5866919)


**Term Frequency-Inverse Document Frequency model (TFIDF)**

It is used to convert text documents to  matrix of  tfidf features.

In [15]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (37500, 5866919)
Tfidf_test: (12500, 5866919)


**Modelling the dataset**

Let us build logistic regression model for both bag of words and tfidf features

In [16]:
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=lr.fit(cv_train_reviews,train_sentiments)
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_reviews,train_sentiments)
print(lr_tfidf)

C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


**Logistic regression model performane on test dataset**

In [17]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(cv_test_reviews)
print(lr_bow_predict)
##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_reviews)
print(lr_tfidf_predict)

[1 1 0 ... 1 1 1]
[0 1 0 ... 0 1 1]


**Accuracy of the model**

In [18]:
#Accuracy score for bag of words
lr_bow_score=accuracy_score(test_sentiments,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(test_sentiments,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

lr_bow_score : 0.74752
lr_tfidf_score : 0.7384


**Print the classification report**

In [19]:
#Classification report for bag of words 
lr_bow_report=classification_report(test_sentiments,lr_bow_predict,target_names=['Positive','Negative'])
print(lr_bow_report)

#Classification report for tfidf features
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['Positive','Negative'])
print(lr_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.73      0.77      0.75      6157
    Negative       0.76      0.73      0.75      6343

   micro avg       0.75      0.75      0.75     12500
   macro avg       0.75      0.75      0.75     12500
weighted avg       0.75      0.75      0.75     12500

              precision    recall  f1-score   support

    Positive       0.70      0.83      0.76      6157
    Negative       0.80      0.65      0.72      6343

   micro avg       0.74      0.74      0.74     12500
   macro avg       0.75      0.74      0.74     12500
weighted avg       0.75      0.74      0.74     12500



**Confusion matrix**

In [20]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,lr_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[4617 1726]
 [1430 4727]]
[[4124 2219]
 [1051 5106]]


**Stochastic gradient descent or Linear support vector machines for bag of words and tfidf features**

In [21]:
#training the linear svm
svm=SGDClassifier(loss='hinge',n_iter=500,random_state=42)
#fitting the svm for bag of words
svm_bow=svm.fit(cv_train_reviews,train_sentiments)
print(svm_bow)
#fitting the svm for tfidf features
svm_tfidf=svm.fit(tv_train_reviews,train_sentiments)
print(svm_tfidf)

C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=500, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)


C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:152: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=500, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=42, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)


**Model performance on test data**

In [22]:
#Predicting the model for bag of words
svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)
#Predicting the model for tfidf features
svm_tfidf_predict=svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


**Accuracy of the model**

In [23]:
#Accuracy score for bag of words
svm_bow_score=accuracy_score(test_sentiments,svm_bow_predict)
print("svm_bow_score :",svm_bow_score)
#Accuracy score for tfidf features
svm_tfidf_score=accuracy_score(test_sentiments,svm_tfidf_predict)
print("svm_tfidf_score :",svm_tfidf_score)

svm_bow_score : 0.4992
svm_tfidf_score : 0.49256


**Print the classification report**

In [24]:
#Classification report for bag of words 
svm_bow_report=classification_report(test_sentiments,svm_bow_predict,target_names=['Positive','Negative'])
print(svm_bow_report)
#Classification report for tfidf features
svm_tfidf_report=classification_report(test_sentiments,svm_tfidf_predict,target_names=['Positive','Negative'])
print(svm_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.50      1.00      0.66      6157
    Negative       0.98      0.01      0.03      6343

   micro avg       0.50      0.50      0.50     12500
   macro avg       0.74      0.51      0.34     12500
weighted avg       0.74      0.50      0.34     12500



C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

    Positive       0.49      1.00      0.66      6157
    Negative       0.00      0.00      0.00      6343

   micro avg       0.49      0.49      0.49     12500
   macro avg       0.25      0.50      0.33     12500
weighted avg       0.24      0.49      0.33     12500



**Plot the confusion matrix**

In [25]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,svm_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,svm_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[  85 6258]
 [   2 6155]]
[[   0 6343]
 [   0 6157]]


**Multinomial Naive Bayes for bag of words and tfidf features**

In [26]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


**Model performance on test data**

In [27]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

[1 1 0 ... 1 1 1]
[1 1 0 ... 1 1 1]


**Accuracy of the model**

In [28]:
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)

mnb_bow_score : 0.74904
mnb_tfidf_score : 0.74864


**Print the classification report**

In [29]:
#Classification report for bag of words 
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.74      0.76      0.75      6157
    Negative       0.76      0.74      0.75      6343

   micro avg       0.75      0.75      0.75     12500
   macro avg       0.75      0.75      0.75     12500
weighted avg       0.75      0.75      0.75     12500

              precision    recall  f1-score   support

    Positive       0.73      0.79      0.75      6157
    Negative       0.77      0.71      0.74      6343

   micro avg       0.75      0.75      0.75     12500
   macro avg       0.75      0.75      0.75     12500
weighted avg       0.75      0.75      0.75     12500



**Plot the confusion matrix**

In [30]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,mnb_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,mnb_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[4673 1670]
 [1467 4690]]
[[4523 1820]
 [1322 4835]]


**Conclusion:**
* We can observed that both logistic regression and multinomial naive bayes model performing well compared to linear support vector  machines.
* Still we can improve the accuracy of the models by preprocessing data and also use deep learning techniques like RNN,LSTM techniques 

**Deep Learning LSTM model using Keras**

In [31]:
# Importing Deep Learning Libraries

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import initializers,regularizers,constraints,optimizers,layers
from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Embedding,Dropout,Activation,GRU,Flatten,Bidirectional
from keras.layers import GlobalMaxPool1D,Convolution1D



Using TensorFlow backend.


*Using the Tokenizer object to convert the text format of the movie reviews into numerical data*

In [57]:
import re
nltk.download()
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

df=pd.read_csv('IMDB_Dataset.csv')


stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df['Processed_Reviews'] =df.review.apply(lambda x: clean_text(x))

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [61]:
#split the dataset  using the train_test split library
X= df.review
y= sentiment_data
train_reviews, test_reviews, train_sentiments, test_sentiments =train_test_split(X,y,test_size=0.25,random_state=42)
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)

(37500,) (37500, 1)
(12500,) (12500, 1)


In [62]:
max_features=6000
tokenizer = Tokenizer(num_words=max_features)
#fit_on_text creates a internal dictionary of tokens specific to this dataset 
tokenizer.fit_on_texts(train_reviews)

X_train = tokenizer.texts_to_sequences(train_reviews)
X_test = tokenizer.texts_to_sequences(test_reviews)


In [67]:
X_train

[[10,
  846,
  11,
  5,
  26,
  32,
  4812,
  627,
  1358,
  493,
  51,
  5375,
  3,
  546,
  108,
  14,
  1,
  1885,
  4,
  3,
  1828,
  1118,
  10,
  118,
  44,
  21,
  998,
  5,
  1108,
  5,
  30,
  220,
  21,
  8,
  3,
  5,
  95,
  18,
  9,
  80,
  620,
  1,
  342,
  7,
  7,
  1308,
  350,
  44,
  243,
  36,
  57,
  109,
  57,
  3,
  1369,
  362,
  9,
  6,
  1739,
  150,
  61,
  43,
  338,
  21,
  1020,
  50,
  72,
  12,
  307,
  1,
  279,
  108,
  45,
  47,
  1421,
  18,
  1,
  848,
  2795,
  1636,
  84,
  1,
  487,
  39,
  216,
  1031,
  1,
  17,
  39,
  115,
  21,
  1666,
  191,
  920,
  1269,
  38,
  1124,
  5,
  385,
  56,
  610,
  7,
  7,
  5,
  26,
  1150,
  10,
  115,
  1627,
  147,
  147,
  16,
  1914,
  5,
  64,
  43,
  233,
  1097,
  9,
  152,
  34,
  3127,
  14,
  9,
  6,
  10,
  581,
  373,
  11,
  17,
  5412,
  9,
  184,
  1014,
  5,
  64,
  1353,
  4813,
  2084,
  14,
  1,
  4263,
  469,
  369,
  3270,
  10,
  430,
  1,
  13,
  190],
 [11,
  6,
  1,
  241,
  4,
  17,

In [68]:
maxlen = 130
X_train = pad_sequences(X_train, maxlen=maxlen)

In [63]:
#Creating the Neural Model
max_features=6000
embed_size = 128
model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         768000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 64)          41216     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 21        
Total params: 810,537
Trainable params: 810,537
Non-trainable params: 0
________________________________________________

In [69]:

batch_size = 20
epochs = 3
model.fit(X_train,train_sentiments, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 30000 samples, validate on 7500 samples
Epoch 1/3


 3660/30000 [==>...........................] - ETA: 2:11:53 - loss: 0.6955 - accuracy: 0.500 - ETA: 1:09:19 - loss: 0.6933 - accuracy: 0.575 - ETA: 47:02 - loss: 0.6946 - accuracy: 0.5167  - ETA: 35:52 - loss: 0.6946 - accuracy: 0.512 - ETA: 29:22 - loss: 0.6941 - accuracy: 0.520 - ETA: 24:50 - loss: 0.6932 - accuracy: 0.541 - ETA: 21:36 - loss: 0.6928 - accuracy: 0.542 - ETA: 19:12 - loss: 0.6937 - accuracy: 0.512 - ETA: 17:20 - loss: 0.6939 - accuracy: 0.505 - ETA: 15:50 - loss: 0.6935 - accuracy: 0.505 - ETA: 14:37 - loss: 0.6935 - accuracy: 0.504 - ETA: 13:37 - loss: 0.6934 - accuracy: 0.508 - ETA: 12:47 - loss: 0.6935 - accuracy: 0.496 - ETA: 12:05 - loss: 0.6933 - accuracy: 0.507 - ETA: 11:26 - loss: 0.6930 - accuracy: 0.520 - ETA: 10:51 - loss: 0.6930 - accuracy: 0.531 - ETA: 10:22 - loss: 0.6930 - accuracy: 0.523 - ETA: 9:56 - loss: 0.6929 - accuracy: 0.527 - ETA: 9:33 - loss: 0.6930 - accuracy: 0.52 - ETA: 9:12 - loss: 0.6930 - accuracy: 0.52 - ETA: 8:54 - loss: 0.6930 - accur

 7380/30000 [======>.......................] - ETA: 3:07 - loss: 0.5895 - accuracy: 0.68 - ETA: 3:07 - loss: 0.5885 - accuracy: 0.68 - ETA: 3:07 - loss: 0.5884 - accuracy: 0.68 - ETA: 3:06 - loss: 0.5881 - accuracy: 0.68 - ETA: 3:06 - loss: 0.5865 - accuracy: 0.68 - ETA: 3:06 - loss: 0.5850 - accuracy: 0.68 - ETA: 3:05 - loss: 0.5850 - accuracy: 0.68 - ETA: 3:05 - loss: 0.5841 - accuracy: 0.68 - ETA: 3:04 - loss: 0.5836 - accuracy: 0.68 - ETA: 3:04 - loss: 0.5824 - accuracy: 0.68 - ETA: 3:03 - loss: 0.5819 - accuracy: 0.68 - ETA: 3:03 - loss: 0.5813 - accuracy: 0.68 - ETA: 3:03 - loss: 0.5809 - accuracy: 0.68 - ETA: 3:02 - loss: 0.5806 - accuracy: 0.68 - ETA: 3:02 - loss: 0.5788 - accuracy: 0.69 - ETA: 3:01 - loss: 0.5788 - accuracy: 0.69 - ETA: 3:01 - loss: 0.5786 - accuracy: 0.69 - ETA: 3:01 - loss: 0.5781 - accuracy: 0.69 - ETA: 3:00 - loss: 0.5770 - accuracy: 0.69 - ETA: 3:00 - loss: 0.5756 - accuracy: 0.69 - ETA: 3:00 - loss: 0.5744 - accuracy: 0.69 - ETA: 2:59 - loss: 0.5739 - ac

11100/30000 [==========>...................] - ETA: 2:25 - loss: 0.4926 - accuracy: 0.75 - ETA: 2:25 - loss: 0.4923 - accuracy: 0.75 - ETA: 2:25 - loss: 0.4920 - accuracy: 0.75 - ETA: 2:24 - loss: 0.4914 - accuracy: 0.75 - ETA: 2:24 - loss: 0.4912 - accuracy: 0.75 - ETA: 2:24 - loss: 0.4920 - accuracy: 0.75 - ETA: 2:24 - loss: 0.4916 - accuracy: 0.75 - ETA: 2:24 - loss: 0.4913 - accuracy: 0.75 - ETA: 2:24 - loss: 0.4909 - accuracy: 0.75 - ETA: 2:23 - loss: 0.4907 - accuracy: 0.75 - ETA: 2:23 - loss: 0.4904 - accuracy: 0.75 - ETA: 2:23 - loss: 0.4902 - accuracy: 0.75 - ETA: 2:23 - loss: 0.4903 - accuracy: 0.75 - ETA: 2:23 - loss: 0.4900 - accuracy: 0.75 - ETA: 2:23 - loss: 0.4908 - accuracy: 0.75 - ETA: 2:23 - loss: 0.4903 - accuracy: 0.75 - ETA: 2:23 - loss: 0.4899 - accuracy: 0.75 - ETA: 2:22 - loss: 0.4895 - accuracy: 0.75 - ETA: 2:22 - loss: 0.4890 - accuracy: 0.75 - ETA: 2:22 - loss: 0.4888 - accuracy: 0.75 - ETA: 2:22 - loss: 0.4886 - accuracy: 0.75 - ETA: 2:22 - loss: 0.4879 - ac

14820/30000 [=============>................] - ETA: 1:57 - loss: 0.4481 - accuracy: 0.78 - ETA: 1:57 - loss: 0.4479 - accuracy: 0.78 - ETA: 1:57 - loss: 0.4479 - accuracy: 0.78 - ETA: 1:56 - loss: 0.4479 - accuracy: 0.78 - ETA: 1:56 - loss: 0.4474 - accuracy: 0.78 - ETA: 1:56 - loss: 0.4474 - accuracy: 0.78 - ETA: 1:56 - loss: 0.4472 - accuracy: 0.78 - ETA: 1:56 - loss: 0.4469 - accuracy: 0.78 - ETA: 1:56 - loss: 0.4469 - accuracy: 0.78 - ETA: 1:56 - loss: 0.4464 - accuracy: 0.78 - ETA: 1:55 - loss: 0.4466 - accuracy: 0.78 - ETA: 1:55 - loss: 0.4464 - accuracy: 0.78 - ETA: 1:55 - loss: 0.4462 - accuracy: 0.78 - ETA: 1:55 - loss: 0.4462 - accuracy: 0.78 - ETA: 1:55 - loss: 0.4461 - accuracy: 0.78 - ETA: 1:55 - loss: 0.4457 - accuracy: 0.78 - ETA: 1:54 - loss: 0.4457 - accuracy: 0.78 - ETA: 1:54 - loss: 0.4454 - accuracy: 0.78 - ETA: 1:54 - loss: 0.4453 - accuracy: 0.78 - ETA: 1:54 - loss: 0.4454 - accuracy: 0.78 - ETA: 1:54 - loss: 0.4454 - accuracy: 0.78 - ETA: 1:54 - loss: 0.4454 - ac

...

26020/30000 [=========================>....] - ETA: 1:07 - loss: 0.1629 - accuracy: 0.93 - ETA: 1:06 - loss: 0.1628 - accuracy: 0.93 - ETA: 1:06 - loss: 0.1628 - accuracy: 0.93 - ETA: 1:06 - loss: 0.1630 - accuracy: 0.93 - ETA: 1:06 - loss: 0.1629 - accuracy: 0.93 - ETA: 1:06 - loss: 0.1629 - accuracy: 0.93 - ETA: 1:06 - loss: 0.1628 - accuracy: 0.93 - ETA: 1:06 - loss: 0.1630 - accuracy: 0.93 - ETA: 1:06 - loss: 0.1629 - accuracy: 0.93 - ETA: 1:05 - loss: 0.1629 - accuracy: 0.93 - ETA: 1:05 - loss: 0.1628 - accuracy: 0.93 - ETA: 1:05 - loss: 0.1628 - accuracy: 0.93 - ETA: 1:05 - loss: 0.1627 - accuracy: 0.93 - ETA: 1:05 - loss: 0.1626 - accuracy: 0.93 - ETA: 1:04 - loss: 0.1626 - accuracy: 0.93 - ETA: 1:04 - loss: 0.1626 - accuracy: 0.93 - ETA: 1:04 - loss: 0.1625 - accuracy: 0.93 - ETA: 1:04 - loss: 0.1624 - accuracy: 0.93 - ETA: 1:04 - loss: 0.1623 - accuracy: 0.93 - ETA: 1:03 - loss: 0.1623 - accuracy: 0.93 - ETA: 1:03 - loss: 0.1624 - accuracy: 0.93 - ETA: 1:03 - loss: 0.1623 - ac

29780/30000 [============================>.] - ETA: 33s - loss: 0.1635 - accuracy: 0.938 - ETA: 33s - loss: 0.1636 - accuracy: 0.938 - ETA: 33s - loss: 0.1637 - accuracy: 0.938 - ETA: 33s - loss: 0.1640 - accuracy: 0.938 - ETA: 33s - loss: 0.1640 - accuracy: 0.938 - ETA: 32s - loss: 0.1640 - accuracy: 0.938 - ETA: 32s - loss: 0.1641 - accuracy: 0.938 - ETA: 32s - loss: 0.1641 - accuracy: 0.938 - ETA: 32s - loss: 0.1640 - accuracy: 0.938 - ETA: 32s - loss: 0.1639 - accuracy: 0.938 - ETA: 32s - loss: 0.1641 - accuracy: 0.938 - ETA: 31s - loss: 0.1644 - accuracy: 0.938 - ETA: 31s - loss: 0.1644 - accuracy: 0.938 - ETA: 31s - loss: 0.1646 - accuracy: 0.938 - ETA: 31s - loss: 0.1647 - accuracy: 0.938 - ETA: 31s - loss: 0.1647 - accuracy: 0.938 - ETA: 30s - loss: 0.1647 - accuracy: 0.938 - ETA: 30s - loss: 0.1648 - accuracy: 0.938 - ETA: 30s - loss: 0.1647 - accuracy: 0.938 - ETA: 30s - loss: 0.1648 - accuracy: 0.938 - ETA: 30s - loss: 0.1648 - accuracy: 0.938 - ETA: 30s - loss: 0.1649 - acc

29980/30000 [============================>.] - ETA: 1s - loss: 0.1683 - accuracy: 0.93 - ETA: 1s - loss: 0.1684 - accuracy: 0.93 - ETA: 1s - loss: 0.1684 - accuracy: 0.93 - ETA: 1s - loss: 0.1685 - accuracy: 0.93 - ETA: 0s - loss: 0.1684 - accuracy: 0.93 - ETA: 0s - loss: 0.1685 - accuracy: 0.93 - ETA: 0s - loss: 0.1685 - accuracy: 0.93 - ETA: 0s - loss: 0.1685 - accuracy: 0.93 - ETA: 0s - loss: 0.1685 - accuracy: 0.93 - ETA: 0s - loss: 0.1684 - accuracy: 0.9369

C:\Users\611242028\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_test_batch_begin) is slow compared to the batch update (0.362242). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


30000/30000 [==============================] - 316s 11ms/step - loss: 0.1683 - accuracy: 0.9369 - val_loss: 0.3647 - val_accuracy: 0.8632


In [70]:
X_test = pad_sequences(X_test, maxlen=maxlen)
prediction = model.predict(X_test)

In [73]:
y_pred = (prediction > 0.5)

In [74]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [ True]])

In [75]:
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(test_sentiments,y_pred)))
print('Confusion matrix:')
confusion_matrix(test_sentiments,y_pred)

F1-score: 0.861068959703254
Confusion matrix:


array([[5745,  412],
       [1236, 5107]], dtype=int64)